In [57]:
import pandas as pd
from nba_api.stats.endpoints import PlayerGameLog
pd.set_option('display.max_colwidth', None) # No limit for column width

In [58]:
PLAYER_ID = 2544  # LeBron James
SEASON = '2020-21'

# 1) Call the NBA API
gamelog = PlayerGameLog(player_id=PLAYER_ID, season=SEASON)
df = gamelog.get_data_frames()[0]

In [60]:
df
print(df.to_string())  # in a standard script/terminal


   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  VIDEO_AVAILABLE
0      22020       2544  0022001072  MAY 16, 2021    LAL @ NOP  W   27   11   22   0.500     2     5    0.400    1    2   0.500     1     0    1    6    3    0    2   0   25          -2                1
1      22020       2544  0022001061  MAY 15, 2021    LAL @ IND  W   28   11   22   0.500     1     4    0.250    1    2   0.500     3     4    7    8    0    0    4   2   24           7                1
2      22020       2544  0022000974  MAY 02, 2021  LAL vs. TOR  L   28    6   11   0.545     2     5    0.400    5    7   0.714     0     7    7    6    0    0    5   2   19          -4                1
3      22020       2544  0022000956  APR 30, 2021  LAL vs. SAC  L   32    6   12   0.500     1     5    0.200    3    5   0.600     1     7    8    7    2    1    5   1   16           5   

In [61]:
def parse_matchup(matchup_str):
    """
    Returns a tuple: (HOME_AWAY, OPPONENT).
    If the string contains 'vs.', it's a Home game.
    If it contains '@', it's an Away game.
    The opponent abbreviation is everything after 'vs.' or '@'.
    """
    if 'vs.' in matchup_str:
        home_away = 'Home'
        opponent = matchup_str.split('vs.')[-1].strip()
    else:  # Then it must have '@'
        home_away = 'Away'
        opponent = matchup_str.split('@')[-1].strip()
    
    return home_away, opponent


In [72]:
df_new = pd.DataFrame()
df_new['Game_ID'] = df['Game_ID']
df_new['HOME_AWAY'], df_new['OPPONENT'] = zip(*df['MATCHUP'].apply(parse_matchup))
df_new['HOME_AWAY'] = df_new['HOME_AWAY'].map({'Home':1, 'Away':0}) #Home = 1 Away = 0
df_new['PTS'] = df['PTS']
avg_pts_by_opp = df_new.groupby('OPPONENT')['PTS'].mean().reset_index(name='AVG_PTS_VS_OPP')
df_new = df_new.merge(avg_pts_by_opp, on='OPPONENT', how='left')
df_new['GAME_DATE'] = df['GAME_DATE']

print(df_new)

       Game_ID  HOME_AWAY OPPONENT  PTS  AVG_PTS_VS_OPP     GAME_DATE
0   0022001072          0      NOP   25       23.000000  MAY 16, 2021
1   0022001061          0      IND   24       21.000000  MAY 15, 2021
2   0022000974          1      TOR   19       19.000000  MAY 02, 2021
3   0022000956          1      SAC   16       16.000000  APR 30, 2021
4   0022000640          1      ATL   10       15.500000  MAR 20, 2021
5   0022000630          1      CHA   37       37.000000  MAR 18, 2021
6   0022000615          1      MIN   25       24.333333  MAR 16, 2021
7   0022000608          0      GSW   22       20.000000  MAR 15, 2021
8   0022000585          1      IND   18       21.000000  MAR 12, 2021
9   0022000542          1      PHX   38       38.000000  MAR 02, 2021
10  0022000527          1      GSW   19       20.000000  FEB 28, 2021
11  0022000512          1      POR   28       28.500000  FEB 26, 2021
12  0022000497          0      UTA   19       19.000000  FEB 24, 2021
13  0022000479      